In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import logging
import sys
import random
import datetime

import cv2
import numpy as np
import scipy as scp
import scipy.misc
import tensorflow as tf

import fcn16_vgg
import utils

In [2]:
RESOURCE = '../dataset'
MODEL_PATH = "./models/model-300-5-40.ckpt"

# Boundary, route, obstacle.
# colors = [[243, 193, 120], [0, 168, 120], [254, 94, 65]]
colors = [[120, 193, 243], [120, 168, 0], [65, 94, 254]]

height = 180
width = 320
num_classes = 3

In [3]:
input_placeholder = tf.placeholder(tf.float32, [None, height, width, num_classes])
output_placeholder = tf.placeholder(tf.float32, [None, height, width, num_classes])

vgg_fcn = fcn16_vgg.FCN16VGG('./vgg16.npy')

with tf.name_scope("content_vgg"):
    vgg_fcn.build(input_placeholder, train=True, num_classes=num_classes)
    
print('Finished building Network.')

npy file loaded
Layer name: conv1_1
Layer shape: (3, 3, 3, 64)
Layer name: conv1_2
Layer shape: (3, 3, 64, 64)
Layer name: conv2_1
Layer shape: (3, 3, 64, 128)
Layer name: conv2_2
Layer shape: (3, 3, 128, 128)
Layer name: conv3_1
Layer shape: (3, 3, 128, 256)
Layer name: conv3_2
Layer shape: (3, 3, 256, 256)
Layer name: conv3_3
Layer shape: (3, 3, 256, 256)
Layer name: conv4_1
Layer shape: (3, 3, 256, 512)
Layer name: conv4_2
Layer shape: (3, 3, 512, 512)
Layer name: conv4_3
Layer shape: (3, 3, 512, 512)
Layer name: conv5_1
Layer shape: (3, 3, 512, 512)
Layer name: conv5_2
Layer shape: (3, 3, 512, 512)
Layer name: conv5_3
Layer shape: (3, 3, 512, 512)
Layer name: fc6
Layer shape: [7, 7, 512, 4096]
Layer name: fc7
Layer shape: [1, 1, 4096, 4096]
Layer name: fc8
Layer shape: [1, 1, 4096, 1000]
Finished building Network.


In [4]:
def regions_to_colored_image(input, colors):
    """Make colored image based on region class.

    Args:
        input: numpy array, int32 - [height, width].
        colors: dictionary of colors - [num_classes].

    Returns:
        new_image: numpy array, int32 - [height, width, 3].
            Colored image.
    """
    height, width = input.shape[:2]

    new_image = np.zeros((height, width, 3), dtype = np.uint8)
    for i in range(len(colors)):
        new_image[input == i] = colors[i]

    return new_image


def merge_images(first_image, second_image, percentage):
    """Merge images.

    Args:
        first_image: numpy array, int32 - [height, width].
        second_image: numpy array, int32 - [height, width].
        percentage: float32.
            Percentage weight of first image.

    Returns:
        merged_image: new_image: numpy array, int32 - [height, width, 3].
    """
    merged_image = first_image * percentage + second_image * (1 - percentage)
    return merged_image.astype(np.uint8)


In [5]:
# Initializing the variables.
init = tf.global_variables_initializer()

# Saver op to save and restore all the variables
saver = tf.train.Saver()

# With CPU mini-batch size can be bigger.
with tf.device('/cpu:0'):
    config = tf.ConfigProto(allow_soft_placement=True)
    config.gpu_options.allow_growth = True

    with tf.Session(config=config) as sess:
        # Restore model weights from previously saved model.
        saver.restore(sess, MODEL_PATH)
        
        cap = cv2.VideoCapture("video0.avi")
        cap.set(1, 0)  # Where frame_no is the frame you want
        while True:
            ret, frame = cap.read()
            frame = cv2.resize(frame, (320, 180))
            
            prediction = sess.run(vgg_fcn.pred_up, feed_dict={input_placeholder: [frame]})
            regions_image = regions_to_colored_image(prediction[0], colors)
            merged_image = merge_images(frame, regions_image, 0.4)   
            view = np.concatenate((frame, merged_image, regions_image), axis=1)
            cv2.imshow('Default View', view)

            k = cv2.waitKey(1) & 0xFF
            if k == 27:
                break

cv2.destroyAllWindows()
        
